# 1회차

- LightGBM 모델은 XGBoost로 대체되었으며 그에 따라 코드가 업데이트됐습니다.
- 또한 랜덤 포레스트의 투표 분류기를 장착하고 XGB의 결과를 RF와 결합합니다.
- 데이터를 한 번 쪼개서 LGBM 조기 정지에 대한 검증 데이터를 사용하는 대신 전체 교육 세트를 훈련할 수 있도록 교육 중 데이터를 분할했습니다. 이것이 이 케이스에서 kfold 분할보다 더 효과가 있음을 발견했습니다.

- 이 커널은 하이퍼 파라미터 최적화를 실행하는 대신 해당 커널의 최적값을 사용하여 더 빨리 실행됩니다.

- __몇 가지 주요 요점은 다음과 같습니다.__:
    - 이 커널은 가장에 대해서만 교육을 실행합니다. 
    - 클래스 크기의 균형을 맞추는 것이 매우 중요해보입니다. 이는 수작업으로 할 수 있고, 언더 샘플링으로 달성할 수 있습니다. 그러나 sklearn API의 LGBM 모델 생성자에 `class_weight='balance'`을 설정하는 것이 가장 간단합니다.
    - 이 커널은 매크로 F1점수를 사용하여 교육을 조기 중단합니다. 
    - 범주는 블라인드 레이블 인코딩 대신 적절한 매핑을 가진 숫자로 바뀝니다.
    - OHE를 라벨 인코딩으로 반전시키면, 트리 모델에 더 적합할 수 있으나 , 논트리 모델에는 안 좋을 수 있으니 유의해야합니다.
    - `idhogar`는 훈련에서 사용되지 않습니다. 정보가 있을 수 있는 유일한 방법은 데이터 누출입니다. 
    - 가구 내에서 집계가 이루어지며 새로운 기능은 수작업으로 제작됩니다. 대부분의 기능이 이미 가정 수준에서 인용되었기 때문에 집계할 수 있는 기능이 많지 않습니다.
    - 투표 분류기는 여러 LGBM 모델에서 평균을 내는데 사용됩니다

### import

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

### mapping

In [2]:
from sklearn.preprocessing import LabelEncoder

# this only transforms the idhogar field, the other things this function used to do are done elsewhere
# 단지 idhogar만을 변환합니다. 이 함수가 사용하던 다른 것들은 다른 곳에서 행해집니다.

def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])
    
# Plot feature importance for sklearn decision trees
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []

    importances = forest.feature_importances_
    
    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # Print the feature ranking
        print('Feature ranking:')
        
    for f in range(X_train.shape[1]):
        if display_results:
            print('%d. feature %d (%f)' % (f+1, indices[f], importances[indices[f]]) + 
                 ' - ' + X_train.columns[indices[f]])
            
            ranked_list.append(X_train.columns[indices[f]])
            
            if importances[indices[f]] == 0.0:
                zero_features.append(X_train.columns[indices[f]])
                
    return ranked_list, zero_features

### feature engineering

In [3]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'),
                ('working_man_fraction', 'r4h2', 'r4t3'),
                ('all_man_fraction', 'r4h3', 'r4t3'),
                ('human_density', 'tamviv', 'rooms'),
                ('human_bed_density', 'tamviv', 'bedrooms'),
                ('rent_per_person', 'v2a1', 'r4t3'),
                ('rent_per_room', 'v2a1', 'rooms'),
                ('mobile_density', 'qmobilephone', 'r4t3'),
                ('tablet_density', 'v18q1', 'r4t3'),
                ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                ('tablet_adult_density', 'v18q1', 'r4t2')]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                ('people_weird_stat', 'tamhog', 'r4t3')]
    
    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
        
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean'],
               'escolari': ['min', 'max', 'mean']}
    
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parenteso', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']
            
    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg
        
    # Drop id's
    df.drop(['Id'], axis=1, inplace=True)
    
    return df

## Read in the data and clean it up

In [4]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

test_ids = test.Id

In [5]:
def process_df(df_):
    # encode the idhogar
    encode_data(df_)
    
    # create aggregate features
    return do_features(df_)

train = process_df(train)
test = process_df(test)

결측치를 정리하고 오브젝트를 숫자형으로 변환합니다.

In [7]:
# Some dependencies are Na, fill those with the square root of the square
train['dependency'] = np.sqrt(train['SQBdependency'])
test['dependency'] = np.sqrt(test['SQBdependency'])

# fill 'no's for education with 0s
train.loc[train['edjefa'] == 'no', 'edjefa'] = 0
train.loc[train['edjefe'] == 'no', 'edjefe'] = 0
test.loc[test['edjefa'] == 'no', 'edjefa'] = 0
test.loc[test['edjefe'] == 'no', 'edjefe'] = 0

# if education is 'yes' and person is head of household, fill with escolari
train.loc[(train['edjefa'] == 'yes') & (train['parentesco1'] == 1), 'edjefa'] = train.loc[(train['edjefa'] == 'yes') & (train['parentesco1'] == 1), 'escolari']
train.loc[(train['edjefe'] == 'yes') & (train['parentesco1'] == 1), 'edjefe'] = train.loc[(train['edjefe'] == 'yes') & (train['parentesco1'] == 1), 'escolari']

test.loc[(test['edjefa'] == 'yes') & (test['parentesco1'] == 1), 'edjefa'] = test.loc[(test['edjefa'] == 'yes') & (test['parentesco1'] == 1), 'escolari']
test.loc[(test['edjefe'] == 'yes') & (test['parentesco1'] == 1), 'edjefe'] = test.loc[(test['edjefe'] == 'yes') & (test['parentesco1'] == 1), 'escolari']

# this field is supposed to be interaction between gender and escolari, but it isn't clear what 'yes' means. let's fill it with 4
train.loc[train['edjefa'] == 'yes', 'edjefa'] = 4
train.loc[train['edjefe'] == 'yes', 'edjefe'] = 4

test.loc[test['edjefa'] == 'yes', 'edjefa'] = 4
test.loc[test['edjefe'] == 'yes', 'edjefe'] = 4

# Convert to int for our models
train['edjefe'] = train['edjefe'].astype('int')
train['edjefa'] = train['edjefa'].astype('int')
test['edjefe'] = test['edjefe'].astype('int')
test['edjefa'] = test['edjefa'].astype('int')

# create feature with max education of either head of household
train['edjef'] = np.max(train[['edjefa', 'edjefe']], axis=1)
test['edjef'] = np.max(test[['edjefa', 'edjefe']], axis=1)

# fill some nas
train['v2a1'] = train['v2a1'].fillna(0)
test['v2a1'] = test['v2a1'].fillna(0)

train['v18q1'] = train['v18q1'].fillna(0)
test['v18q1'] = test['v18q1'].fillna(0)

train['rez_esc'] = train['rez_esc'].fillna(0)
test['rez_esc'] = test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnull(), 'meaneduc'] = 0
train.loc[train.SQBmeaned.isnull(), 'SQBmeaned'] = 0

test.loc[test.meaneduc.isnull(), 'meaneduc'] = 0
test.loc[test.SQBmeaned.isnull(), 'SQBmeaned'] = 0

# fix some inconsistencies in the data - some rows indicate both that the household does and does not not have a toilet,
# if there is no water we'll assume they do not
train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), 'v14a'] = 0
train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), 'sanitario'] = 0

test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), 'v14a'] = 0
test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), 'sanitario'] = 0

In [8]:
def train_test_apply_func(train_, test_, func_):
    test_['Target'] = 0
    xx = pd.concat([train_, test_])
    
    xx_func = func_(xx)
    train_ = xx_func.iloc[:train_.shape[0], :]
    test_ = xx_func.iloc[train_.shape[0]:, :].drop('Target', axis=1)
    
    del xx, xx_func
    return train_, test_
    

In [9]:
# convert the one hot fields into label encoded
train, test = train_test_apply_func(train, test, convert_OHE2LE)

NameError: name 'convert_OHE2LE' is not defined